In [69]:
import os
import pandas as pd

# Set the working directory
os.chdir("C:\TradeGroup")
my_wd = os.getcwd()

#1402
#file_path_Imp = f"{my_wd}/Data/Cleaned_Data/Y1402Imports.xlsx"
#file_path_Exp = f"{my_wd}/Data/Cleaned_Data/Y1402Exports.xlsx"

#1403 First mounth
file_path_Imp = f"{my_wd}/Data/Cleaned_Data/Y1403Imports.xlsx"
file_path_Exp = f"{my_wd}/Data/Cleaned_Data/Y1403Exports.xlsx"


In [70]:
def create_folder(folder_path):
    if not os.path.exists(f'{new_directory}/{folder_path}'):
        os.makedirs(f'{new_directory}/{folder_path}')
    else:
        print(f'Already Exists')

In [71]:
Imports = pd.read_excel(file_path_Imp,sheet_name=0)
Exports = pd.read_excel(file_path_Exp,sheet_name=0)
#Imports['Year']=Imports['Year'].astype('int')
#Exports['Year']=Exports['Year'].astype('int')




In [65]:
Imports

,Year,Month,Custom,Country,Tcode,Tdesc,Weight,Rials,Dollars
0,1402,1,منطقه ويژه اقتصادي پيام,امارات متحده عربي,30021210,آنتی سرم ها و سایر مشتقات خونی با کاربرد پزشکی,1345,367099249734,1288068
1,1402,1,منطقه ويژه اقتصادي پيام,امارات متحده عربي,34039990,ساير فرآورده هاي روان كننده فاقد روغن نفتي يا ...,1920,6019200000,21120
2,1402,1,منطقه ويژه اقتصادي پيام,امارات متحده عربي,38221920,کیت آزمایشگاهی با کاربرد پزشکی,7242,749461246640,2629689
3,1402,1,منطقه ويژه اقتصادي پيام,امارات متحده عربي,39269030,مصنوعات از مواد پلاستيكي جهت مصارف پزشکي,1996,115015140785,403562
4,1402,1,منطقه ويژه اقتصادي پيام,امارات متحده عربي,40113000,لاستيك رويي چرخ، بادي نو، از كائوچو از انواع م...,5506,16743750000,58750
...,...,...,...,...,...,...,...,...,...
185658,1402,9,منطقه ويژه اقتصادي سلفچگان,ترکيه,87021000,ک تنها با موتور پيستوني درونسوز تراكمي ک احترا...,4000,36114048000,126716
185659,1402,9,منطقه ويژه اقتصادي سلفچگان,ترکيه,87032290,ساير وسايل نقليه موتوري با حجم سيلندر بيشتر از...,4955,34035228000,119422
185660,1402,9,منطقه ويژه اقتصادي سلفچگان,ترکيه,96032900,فرچه ریش تراش ، بروس مو ، بروس ناخن ، بروس مژه...,3,828840078,2908
185661,1402,9,منطقه ويژه اقتصادي سلفچگان,ترکيه,96161000,عطرپاش و پاشنده هاي آرايشي همانند و تجهيزات و ...,99,917851452,3221


In [72]:
Imports = Imports.rename(columns={
                                  "Weight": "Weight_Imp", "Rials":"Rials_Imp","Dollars": "Dollars_Imp"})

Exports = Exports.rename(columns={
                                  "Weight": "Weight_Exp", "Rials":"Rials_Exp","Dollars": "Dollars_Exp"})

unique_months = Imports['Month'].unique()

In [74]:
new_directory = my_wd
create_folder('Data/Monthly_Data/Detailed')
DataFrames =[]
unique_months = unique_months[~pd.isna(unique_months)]
for month_value in unique_months:
    # Create a new DataFrame for each month
    df_name_Imp = f'Imports{int(month_value):02d}'
    df_name_Exp = f'Exports{int(month_value):02d}'
    df_name = f'M{int(month_value):02d}'
    
    # Filter rows for the current month
    globals()[df_name_Imp] = Imports[Imports['Month'] == month_value].reset_index(drop=True)
    globals()[df_name_Exp] = Exports[Exports['Month'] == month_value].reset_index(drop=True)
    globals()[df_name]     = pd.merge(globals()[df_name_Exp],globals()[df_name_Imp],
                                      on=['Year','Month','Custom','Country','Tcode','Tdesc'],
                                      how='outer')
    globals()[df_name]=globals()[df_name].fillna(0)
    DataFrames.append(df_name)
    #Change year based on the year
    xlsx_filename = os.path.join(my_wd, f'Data/Monthly_Data/Detailed/M1403{int(month_value):02d}Detailed.xlsx')
    globals()[df_name].to_excel(xlsx_filename, index=False)    
    


Already Exists


In [75]:
for df in DataFrames:
    M_monthly= globals()[df].groupby(['Tcode','Tdesc']).agg(
        Tcode=('Tcode','first'),
        Tdesc=('Tdesc','first'),
        Weight_Imp=('Weight_Imp','sum'),
        Rials_Imp=('Rials_Imp','sum'),
        Dollars_Imp=('Dollars_Imp','sum'),
          Weight_Exp=('Weight_Exp','sum'),
        Rials_Exp=('Rials_Exp','sum'),
        Dollars_Exp=('Dollars_Exp','sum')
    ).reset_index(drop=True)
    xlsx_filename_Sum = os.path.join(my_wd, f'Data/Monthly_Data//M1403{df}.xlsx')
    M_monthly.to_excel(xlsx_filename_Sum, index=False)

    